# 反爬：登入授權模擬

* 了解「登入權限機制」的反爬蟲機制
* 「登入權限機制」反爬蟲的因應策略

## 作業目標

* 找一個需要登入的網站試試看，並說明思考流程
（如果不知道要用哪個網站的話，可以試試看 https://github.com/new 網址，未登入時會被導向登入頁）



In [1]:
import requests
from bs4 import BeautifulSoup

# 1. 在登入頁中取得 token 和 cookies

In [3]:
response_login = requests.get('https://github.com/login')
soup = BeautifulSoup(response_login.text, 'html.parser')
token = soup.find('input', {'name':'authenticity_token'})['value']
print(token)

jRghNazxBV0EgTWupEIaE1rkcMLm7phpEvcMeXLuVKRzxkFEeJib+tUDoYCWVSBhAdChqlX1BVtCsQXa8SmNpQ==


In [4]:
cookies_login = response_login.cookies.get_dict()
print(cookies_login)

{'_octo': 'GH1.1.1558640360.1605064704', 'logged_in': 'no', '_gh_sess': 'z2EOXhxlDoiY2gijR3WeCJ52An%2FM%2BdC7umqcyVQU%2BgfZC60Oqc6p9QVMiOMUCnm3e9HGwoT8t2Wnb0zTEeowy6SPSzn9EQh%2BeBDO%2F26a7om4A98j%2FW5ADrqab%2B85X3xpgld2R0jXWzCA8Gordm8JIWAUfk8dWMy10tCBiMBhqwDpzd%2BmiJpsYT2wFyIqTI4MgKaRjuAw4TmcAEyGMylCIr2YDxYwmB7JbCkYcoCVRlM%2Fg1oiAfa9iQ819%2BAmOUW1aqgIzHbIRmNHFNiC1Dg9ZQ%3D%3D--HCzj021ZbCreYioD--gfVMICBYKudI5bs9nQ%2FroA%3D%3D'}


# 2. 先輸入錯誤帳號密碼取得轉接的頁面網址和需要帶入的資訊(包括正確的帳密)並在 post 之後取得新的 cookies

In [5]:
response_session = requests.post('https://github.com/session',
                                data = {'commit':'Sign in',
                                        'authenticity_token':token,
                                        'login':'brianooo',
                                        'password': 'ANNlol@0521'},
                                cookies = cookies_login)
cookies_session = response_session.cookies.get_dict()
print(cookies_session)

{'logged_in': 'yes', '__Host-user_session_same_site': 'yZWsOgLF-aDKZbrRECdFN3zve8fok2Xh3nSgk9qbtW2I_Ps9', '_gh_sess': 'ZW%2BMwCx76pSDK%2Fs45J7beyzhEGbr%2FtD1BNpJ8TkEB7EptNQMzzbh9qWnRfxG0IqrvHfQoRj0NS2%2Fy1rCmamTDPGRMaLogXGqBNhiMbkd9j3iMlmwV1TelH4sBAWNHF2923NX5m8ybP1CmMQl5Jg%2BBgRLrGbcKgIuj%2BPdKay9hif5ZBfPg9ILI9VWZZsW5mYj%2FLNNexL3l9fXLpF1Uz2%2Fc0Ue9U5HUsU%2FojwELtG57WtyMDI62tyMMqAL0uk9vhsxKYPg4ZyjQRQpbA5tu109UJjnbmjhI6kNINXZ5u8WSreajK7I--SNK9Rr0oW1V2EP7p--HW6c3x%2F6OhOP6se%2BIu9udg%3D%3D', 'has_recent_activity': '1', 'user_session': 'yZWsOgLF-aDKZbrRECdFN3zve8fok2Xh3nSgk9qbtW2I_Ps9'}


# 3. 帶入新的 cookies 就能獲取等同於在瀏覽器中輸入帳密所進入的帳號頁面

In [18]:
response = requests.get('https://github.com/', cookies = cookies_session)
soup = BeautifulSoup(response.text, 'html.parser')
summary = soup.find_all('details')[1]
print('Summary of Login Info: ', summary.prettify())
print()
print('Login Account: ')
print(summary.find('img')['alt'])


<details class="details-overlay details-reset js-feature-preview-indicator-container" data-feature-preview-indicator-src="/users/brianooo/feature_preview/indicator_check">
<summary aria-label="View profile and more" class="Header-link" data-ga-click="Header, show menu, icon:avatar">
<img alt="@brianooo" class="avatar avatar-user" height="20" src="https://avatars3.githubusercontent.com/u/52178459?s=60&amp;v=4" width="20">
<span class="feature-preview-indicator js-feature-preview-indicator" hidden="" style="top: 1px;"></span>
<span class="dropdown-caret"></span>
</img></summary>
<details-menu class="dropdown-menu dropdown-menu-sw" preload="" src="/users/52178459/menu" style="width: 180px">
<include-fragment>
<p class="text-center mt-3" data-hide-on-error="">
<img alt="Loading…" height="32" src="https://github.githubassets.com/images/spinners/octocat-spinner-64.gif" width="32">
</img></p>
<p class="ml-1 mb-2 mt-2 text-gray-dark" data-show-on-error="">
<svg aria-hidden="true" class="octico